# Comprobación del gradiente

Bienvenidos a la última tarea de esta semana. En esta tarea implementarás la comprobación de gradientes.

Al final de este cuaderno, serás capaz de:

Implementar la comprobación de gradientes para verificar la exactitud de tu implementación de backprop.

## Nota importante sobre el envío al AutoGrader

Antes de enviar su tarea al AutoGrader, por favor asegúrese de que no está haciendo lo siguiente:

1. 1. No ha añadido ninguna declaración _extra_ `print` en la tarea.
2. 2. No ha añadido ninguna celda de código _extra_ en la tarea.
3. No ha cambiado ningún parámetro de la función.
4. No ha utilizado ninguna variable global dentro de sus ejercicios calificados. A menos que se le indique específicamente que lo haga, por favor absténgase de hacerlo y utilice las variables locales en su lugar.
5. 5. No está cambiando el código de asignación donde no es necesario, como la creación de variables _extra_.

Si hace algo de lo siguiente, obtendrá un error como `Grader no encontrado` (o similarmente inesperado) al enviar su tarea. Antes de pedir ayuda/depurar los errores de su tarea, compruebe esto primero. Si este es el caso, y no recuerda los cambios que ha realizado, puede obtener una nueva copia de la tarea siguiendo estas [instrucciones](https://www.coursera.org/learn/deep-neural-network/supplement/QWEnZ/h-ow-to-refresh-your-workspace).

## Table of Contents
- [1 - Packages](#1)
- [2 - Problem Statement](#2)
- [3 - How does Gradient Checking work?](#3)
- [4 - 1-Dimensional Gradient Checking](#4)
    - [Exercise 1 - forward_propagation](#ex-1)
    - [Exercise 2 - backward_propagation](#ex-2)
    - [Exercise 3 - gradient_check](#ex-3)
- [5 - N-Dimensional Gradient Checking](#5)
    - [Exercise 4 - gradient_check_n](#ex-4)

In [4]:
import numpy as np
from testCases import *
from public_tests import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

%load_ext autoreload
%autoreload 2

<a name='2'></a>
## 2 - Planteamiento del problema

Formas parte de un equipo que trabaja para que los pagos por móvil estén disponibles en todo el mundo, y se te pide que construyas un modelo de aprendizaje profundo para detectar el fraude: cada vez que alguien hace un pago, quieres ver si el pago puede ser fraudulento, como por ejemplo si la cuenta del usuario ha sido tomada por un hacker.

Ya sabes que la retropropagación es bastante difícil de implementar, y a veces tiene errores. Como se trata de una aplicación de misión crítica, el director general de su empresa quiere estar realmente seguro de que su implementación de la retropropagación es correcta. Tu director general dice: "¡Dame pruebas de que tu retropropagación funciona realmente!". Para dar esta seguridad, vas a utilizar la "comprobación de gradiente".

¡Vamos a hacerlo!

pero tenemos que encontrar una manera par<a name='3'></a>
## 3 - ¿Cómo funciona la comprobación de gradientes?
La propagación hacia atrás calcula los gradientes $\frac{\partial J}{\partial \theta}$, donde $\theta$ denota los parámetros del modelo. $J$ se calcula utilizando la propagación hacia adelante y su función de pérdida.

Debido a que la propagación hacia adelante es relativamente fácil de implementar, usted está seguro de que lo hizo bien, y por lo que son casi 100% seguro de que está calculando el costo $J $ correctamente. Por lo tanto, usted puede utilizar su código para el cálculo de $ J $ para verificar el código para el cálculo de $\frac{\partial J}{\partial \theta}$.

Volvamos a la definición de una derivada (o gradiente):$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

Si no estás familiarizado con la notación "$\displaystyle \lim_{\varepsilon \to 0}$", es sólo una forma de decir "cuando $\varepsilon$ es muy, muy pequeño".

Usted sabe lo siguiente:


$\frac{\partial J}{\partial \theta}$ es lo que quieres para asegurarte de que estás calculando correctamente.
You can compute $J(\theta + \varepsilon)$ and $J(\theta - \varepsilon)$ (en el caso de que $\theta$ isea un número real), ya que estás seguro de que tu implementación para $J$ es correcta.
tilicemos la ecuación (1) y un valor pequeño para $\varepsilon$ para convencer a su director general de que su código para calcular $\frac{\partial J}{\partial \theta}$ es correcto!

<a name='4'></a>
## 4 - Comprobación del gradiente en 1 dimensión

Consideremos una función lineal 1D $J(\theta) = \theta x$. El modelo contiene un solo parámetro de valor real $\theta$, y toma $x$ como entrada.

Se implementará un código para calcular $J(.)$ y su derivada $\frac{parcial J}{parcial \theta}$. A continuación, utilizará la comprobación del gradiente para asegurarse de que el cálculo de la derivada de $J$ es correcto. 

<img src="images/1Dgrad_kiank.png" style="width:600px;height:250px;">
<caption><center><font color='purple'><b>Figure 1</b>:1D linear model </font></center></caption>

El diagrama anterior muestra los pasos clave del cálculo: Primero empieza con $x$, luego evalúa la función $J(x)$ ("propagación hacia delante"). A continuación, calcular la derivada $\frac{{parcial J}{parcial \theta}$ ("propagación hacia atrás"). 

<a name='ex-1'></a>
### Exercise 1 - forward_propagation

Implementa la `propagación hacia adelante`. Para esta función simple calcula $J(.)$

In [5]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(x, theta):
    """
    Implementa la propagación lineal hacia adelante (calcula J) presentada en la Figura 1 (J(theta) = theta * x)
    
    Argumentos:
    x -- una entrada de valor real
    theta -- nuestro parámetro, un número real también
    
    Devuelve:
    J -- el valor de la función J, calculado mediante la fórmula J(theta) = theta * x
    """
    
    # (aprox. 1 línea)
    # J = = theta * x
    # YOUR CODE STARTS HERE
    J = theta * x
    
    # YOUR CODE ENDS HERE
    
    return J

In [6]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))
forward_propagation_test(forward_propagation)

J = 8
 All tests passed.


<a name='ex-2'></a>
### Ejercicio 2 - propagación_hacia_atrás

Ahora, implementa el paso de `propagación hacia atrás` (cálculo de la derivada) de la Figura 1. Es decir, calcula la derivada de $J(\theta) = \theta x$ con respecto a $\theta$. Para ahorrarte el cálculo, deberías obtener $dtheta = \frac { \partial J }{ \partial \theta} = x$.

In [7]:
# GRADED FUNCTION: backward_propagation

def backward_propagation(x, theta):
    """
    Calcula la derivada de J con respecto a theta (ver Figura 1).
    
    Argumentos:
    x -- una entrada de valor real
    theta -- nuestro parámetro, un número real también
    
    Devuelve:
    dtheta -- el gradiente del coste con respecto a theta
    """
    
    # (approx. 1 line)
    # dtheta = 
    # YOUR CODE STARTS HERE
    
    dtheta = x
    # YOUR CODE ENDS HERE
    
    return dtheta

In [8]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))
backward_propagation_test(backward_propagation)

dtheta = 2
 All tests passed.


<a name='ex-3'></a>
### Ejercicio 3 - gradient_check

Para demostrar que la función `propagación_hacia_atrás()` está calculando correctamente el gradiente $\frac{{parcial J}{parcial \theta}$, vamos a implementar la comprobación del gradiente.

**Instrucciones**:
- Primero calcule "gradapprox" usando la fórmula anterior (1) y un valor pequeño de $\varepsilon$. Aquí están los pasos a seguir:
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}{2 \varepsilon}$
- A continuación, calcular el gradiente utilizando la propagación hacia atrás, y almacenar el resultado en una variable "grad"
- Por último, calcular la diferencia relativa entre "gradapprox" y el "grad" utilizando la siguiente fórmula

$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2} $$

Necesitarás 3 pasos para calcular esta fórmula:
   - 1'. calcular el numerador utilizando np.linalg.norm(...)
   - 2'. calcular el denominador. Tendrás que llamar a np.linalg.norm(...) dos veces.
   - 3'. dividirlos.
- Si esta diferencia es pequeña (digamos menos de $10^{-7}$), puedes estar bastante seguro de que has calculado tu gradiente correctamente. De lo contrario, puede haber un error en el cálculo del gradiente. 

In [9]:
# GRADED FUNCTION: gradient_check

def gradient_check(x, theta, epsilon=1e-7, print_msg=False):
    """
    Implement the backward propagation presented in Figure 1.
    
    Arguments:
    x -- a float input
    theta -- our parameter, a float as well
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient. Float output
    """
    
    # Compute gradapprox using left side of formula (1). epsilon is small enough, you don't need to worry about the limit.
    # (approx. 5 lines)
    # theta_plus =                                # Step 1
    # theta_minus =                               # Step 2
    # J_plus =                                    # Step 3
    # J_minus =                                   # Step 4
    # gradapprox =                                # Step 5
    # YOUR CODE STARTS HERE
    theta_plus  = theta + epsilon                # Step 1
    theta_minus = theta - epsilon                # Step 2
    J_plus      = theta_plus * x                 # Step 3
    J_minus     = theta_minus * x                # Step 4
    gradapprox  = (J_plus - J_minus)/(2*epsilon) # Step 5
    
    # YOUR CODE ENDS HERE
    
    # Check if gradapprox is close enough to the output of backward_propagation()
    #(approx. 1 line) DO NOT USE "grad = gradapprox"
    # grad =
    # YOUR CODE STARTS HERE
    grad = backward_propagation(x, theta)
    
    # YOUR CODE ENDS HERE
    
    #(approx. 3 lines)
    # numerator =                                 # Step 1'
    # denominator =                               # Step 2'
    # difference =                                # Step 3'
    # YOUR CODE STARTS HERE
    numerator   = np.linalg.norm(grad-gradapprox)                                # Step 1
    denominator = np.linalg.norm(grad)+ np.linalg.norm(gradapprox)             # Step 2
    difference  = numerator/denominator                                         # Step 3
    
    # YOUR CODE ENDS HERE
    if print_msg:
        if difference > 2e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [10]:
x, theta = 2, 4
difference = gradient_check(x, theta, print_msg=True)

Your backward propagation works perfectly fine! difference = 2.919335883291695e-10


Felicidades, la diferencia es menor que el umbral de $10^{-7}$. Así que usted puede tener una alta confianza de que usted ha calculado correctamente el gradiente en `propagación hacia atrás() `. 

Ahora, en el caso más general, su función de coste $J$ tiene más de una sola entrada 1D. ¡Cuando usted está entrenando una red neuronal, $\theta$ en realidad consiste en múltiples matrices $W^[l]}$ y sesgos $b^[l]}$! Es importante saber cómo hacer una comprobación de gradiente con entradas de mayor dimensión. ¡Hagámoslo!

<a name='5'></a>
## 5 - N-Dimensional Gradient Checking

la siguiente figura describe la propagación hacia delante y hacia atrás de su modelo de detección de fraude.

<img src="images/NDgrad_kiank.png" style="width:600px;height:400px;">
<caption><center><font color='purple'><b>Figure 2</b>: Deep neural network. LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID</font></center></caption>

Veamos sus implementaciones para la propagación hacia adelante y hacia atrás. 

In [11]:
def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation (and computes the cost) presented in Figure 3.
    
    Arguments:
    X -- training set for m examples
    Y -- labels for m examples 
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    
    Returns:
    cost -- the cost function (logistic cost for one example)
    cache -- a tuple with the intermediate values (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

    """
    
    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    log_probs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(log_probs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

Ahora, ejecuta la propagación hacia atrás.

In [12]:
def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T) * 2
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 4. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

Ha obtenido algunos resultados en el conjunto de pruebas de detección de fraudes, pero no está 100% seguro de su modelo. Nadie es perfecto. Implementemos la comprobación de gradientes para verificar si sus gradientes son correctos.

**¿Cómo funciona la comprobación del gradiente?

Al igual que en las secciones 3 y 4, se quiere comparar "gradapprox" con el gradiente calculado por retropropagación. La fórmula sigue siendo:

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

Sin embargo, $\theta$ ya no es un escalar. Es un diccionario llamado "parámetros". La función "`diccionario_a_vector()`" ha sido implementada para usted. Convierte el diccionario "parámetros" en un vector llamado "valores", que se obtiene al reestructurar todos los parámetros (W1, b1, W2, b2, W3, b3) en vectores y concatenarlos.

La función inversa es "`vector_a_diccionario`" que devuelve el diccionario de "parámetros".

<img src="images/dictionary_to_vector.png" style="width:600px;height:400px;">
<caption><center><font color='purple'><b>Figure 2</b>: dictionary_to_vector() y vector_to_dictionary(). Necesitará estas funciones en gradient_check_n()</font></center></caption>

El diccionario "gradients" también ha sido convertido en un vector "grad" usando gradients_to_vector(), así que no necesitas preocuparte por eso.

Ahora, para cada parámetro de tu vector, aplicarás el mismo procedimiento que para el ejercicio gradient_check. Almacenarás cada aproximación del gradiente en un vector `gradapprox`. Si la comprobación va como se espera, cada valor de esta aproximación debe coincidir con los valores reales del gradiente almacenados en el vector `grad`. 

Ten en cuenta que `grad` se calcula utilizando la función `gradients_to_vector`, que utiliza las salidas de los gradientes de la función `backward_propagation_n`.

<a name='ex-4'></a>
### Ejercicio 4 - gradient_check_n

Implementa la función de abajo.

**Instrucciones**: Aquí hay un pseudocódigo que te ayudará a implementar la comprobación del gradiente.

Para cada i en num_parámetros:
- Para calcular `J_plus[i]`:
    1. Establecer $\theta^{+}$ a `np.copy(parameters_values)`
    2. Establecer $\theta^{+}_i$ a $\theta^{+}_i + \varepsilon$
    3. Calcular $J^{+}_i$ utilizando a `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.     
- Para calcular `J_minus[i]`: hacer lo mismo con $\theta^{-}$
- Calcula $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

Así, se obtiene un vector gradapprox, donde gradapprox[i] es una aproximación del gradiente con respecto a `valores_de_parámetro[i]`. Ahora puedes comparar este vector gradapprox con el vector de gradientes de la retropropagación. Al igual que para el caso 1D (Pasos 1', 2', 3'), calcular: 
$$ diferencia = \frac {\| grad - gradapprox |_2}{\| grad |_2 + \| gradapprox |_2 } |tag{3}$$

**Nota**: Utilice `np.linalg.norm` para obtener las normas

In [15]:
# GRADED FUNCTION: gradient_check_n

def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7, print_msg=False):
    """
    Comprueba si backward_propagation_n calcula correctamente el gradiente de la salida del coste por forward_propagation_n
    
    Argumentos:
    parameters -- diccionario python que contiene sus parámetros "W1", "b1", "W2", "b2", "W3", "b3":
    grad -- salida de backward_propagation_n, contiene los gradientes del coste con respecto a los parámetros. 
    x -- punto de datos de entrada, de forma (tamaño de entrada, 1)
    y -- verdadera "etiqueta"
    epsilon -- pequeño desplazamiento de la entrada para calcular el gradiente aproximado con la fórmula(1)
    
    Devuelve:
    diferencia -- diferencia (2) entre el gradiente aproximado y el gradiente de propagación hacia atrás
    """
    
    # Set-up variables
    parameters_values, _ = dictionary_to_vector(parameters)
    
    grad           = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus         = np.zeros((num_parameters, 1))
    J_minus        = np.zeros((num_parameters, 1))
    gradapprox     = np.zeros((num_parameters, 1))
    
    # Compute gradapprox
    for i in range(num_parameters):
        
        # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
        # "_" is used because the function you have to outputs two parameters but we only care about the first one
        #(approx. 3 lines)
        # theta_plus =                                        # Step 1
        # theta_plus[i] =                                     # Step 2
        # J_plus[i], _ =                                     # Step 3
        # YOUR CODE STARTS HERE
        theta_plus    = np.copy(parameters_values)                                     
        theta_plus[i] = theta_plus[i] + epsilon                                  
        J_plus[i], _  = forward_propagation_n( X, Y, vector_to_dictionary( theta_plus ))
        
        # YOUR CODE ENDS HERE
        
        # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
        #(approx. 3 lines)
        # theta_minus =                                    # Step 1
        # theta_minus[i] =                                 # Step 2        
        # J_minus[i], _ =                                 # Step 3
        # YOUR CODE STARTS HERE
        theta_minus    = np.copy(parameters_values)                            
        theta_minus[i] = theta_minus[i] - epsilon                                 
        J_minus[i], _  = forward_propagation_n( X, Y, vector_to_dictionary( theta_minus ))
        
        # YOUR CODE ENDS HERE
        
        # Compute gradapprox[i]
        # (approx. 1 line)
        # gradapprox[i] = 
        # YOUR CODE STARTS HERE
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
        
        # YOUR CODE ENDS HERE
    
    # Compare gradapprox to backward propagation gradients by computing difference.
    # (approx. 3 line)
    # numerator =                                             # Step 1'
    # denominator =                                           # Step 2'
    # difference =                                            # Step 3'
    # YOUR CODE STARTS HERE
    numerator   = np.linalg.norm( ( grad-gradapprox )) 
    denominator = np.linalg.norm( grad ) + np.linalg.norm( gradapprox )
    difference  = numerator / denominator 
    
    # YOUR CODE ENDS HERE
    if print_msg:
        if difference > 2e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")

    return difference

In [16]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y, 1e-7, True)
expected_values = [0.2850931567761623, 1.1890913024229996e-07]
assert not(type(difference) == np.ndarray), "You are not using np.linalg.norm for numerator or denominator"
assert np.any(np.isclose(difference, expected_values)), "Wrong value. It is not one of the expected values"


There is a mistake in the backward propagation! difference = 0.2850931567761623


**Expected output**:

<table>
    <tr>
        <td>  <b> There is a mistake in the backward propagation!</b>  </td>
        <td> difference = 0.2850931567761623 </td>
    </tr>
</table>

¡Parece que había errores en el código de `backward_propagation_n`! Menos mal que has implementado la comprobación del gradiente. Vuelve a `propagación_hacia_atrás` y trata de encontrar/corregir los errores *(Pista: comprueba dW2 y db1)*. Vuelve a ejecutar la comprobación del gradiente cuando creas que lo has solucionado. Recuerda que tendrás que volver a ejecutar la celda que define `propagación_hacia_atrás_n()` si modificas el código. 

¿Puedes conseguir que la comprobación del gradiente declare que tu cálculo de la derivada es correcto? Aunque esta parte de la tarea no se califica, deberías intentar encontrar el error y volver a ejecutar la comprobación de gradiente hasta que estés convencido de que la retropropagación está ahora correctamente implementada. 

**Notas** 
- La comprobación del gradiente es lenta. Aproximar el gradiente con $\frac{\partial J}{\partial \theta} \approx \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$ es computacionalmente costoso. Por esta razón, no ejecutamos la comprobación del gradiente en cada iteración durante el entrenamiento. Sólo unas pocas veces para comprobar si el gradiente es correcto. 
- La comprobación del gradiente, al menos como la hemos presentado, no funciona con el abandono. Por lo general, se ejecuta el algoritmo de comprobación de gradiente sin dropout para asegurarse de que su backprop es correcto, a continuación, añadir dropout. 

Enhorabuena. Now you can be confident that your deep learning model for fraud detection is working correctly! You can even use this to convince your CEO. :) 
<br>
<font color='blue'>
    
**Lo que debes recordar de este cuaderno**:
- La comprobación del gradiente verifica la proximidad entre los gradientes de la retropropagación y la aproximación numérica del gradiente (calculada mediante la propagación hacia delante).
- La comprobación del gradiente es lenta, por lo que no es conveniente ejecutarla en cada iteración del entrenamiento. Por lo general, se ejecuta sólo para asegurarse de que el código es correcto, luego se desactiva y se utiliza backprop para el proceso de aprendizaje real. 